## Data Exploration

In [385]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
%matplotlib qt
#from skimage.feature import hog
#from skimage import color, exposure
# images are divided up into vehicles and non-vehicles

images = glob.glob('./datas/*.jpeg')
cars = []
notcars = []

for image in images:
    if 'image' in image or 'extra' in image:
        notcars.append(image)
    else:
        cars.append(image)
        
# Define a function to return some characteristics of the dataset 
# Define a function to return some characteristics of the dataset 
def data_look(car_list, notcar_list):
    data_dict = {}
    # Define a key in data_dict "n_cars" and store the number of car images
    data_dict["n_cars"] = len(car_list)
    # Define a key "n_notcars" and store the number of notcar images
    data_dict["n_notcars"] = len(notcar_list)
    # Read in a test image, either car or notcar
    example_img = mpimg.imread(car_list[0])
    # Define a key "image_shape" and store the test image shape 3-tuple
    data_dict["image_shape"] = example_img.shape
    # Define a key "data_type" and store the data type of the test image.
    data_dict["data_type"] = example_img.dtype
    # Return data_dict
    return data_dict
    
data_info = data_look(cars, notcars)

print('Your function returned a count of', 
      data_info["n_cars"], ' cars and', 
      data_info["n_notcars"], ' non-cars')
print('of size: ',data_info["image_shape"], ' and data type:', 
      data_info["data_type"])
# Just for fun choose random car / not-car indices and plot example images   
car_ind = np.random.randint(0, len(cars))
notcar_ind = np.random.randint(0, len(notcars))
    
# Read in car / not-car images
car_image = mpimg.imread(cars[car_ind])
notcar_image = mpimg.imread(notcars[notcar_ind])


# Plot the examples
fig = plt.figure()
plt.subplot(121)
plt.imshow(car_image)
plt.title('Example Car Image')
plt.subplot(122)
plt.imshow(notcar_image)
plt.title('Example Not-car Image')

Your function returned a count of 1196  cars and 1125  non-cars
of size:  (64, 64, 3)  and data type: uint8


## HOG Algorithm

In [4]:
from skimage.feature import hog

#Use HOG algorithm
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False,
                                  visualise=True, feature_vector=False)
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False,
                       visualise=False, feature_vector=feature_vec)
        return features

## Color convert

In [5]:
def convert_color_size(img, color_space='RGB', size=(32, 32)):
    # Convert image to new color space (if specified)
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)             
    # Use cv2.resize()
    output = cv2.resize(feature_image, size)
    # Return image
    return output

## Binary Spatial

In [11]:
def bin_spatial(img, color_space='HSV', size=(32, 32)):
    # Convert image to new color space(HSV)
    feature_image = convert_color_size(img, color_space, size)           
    # Use only S, V channel
    color2 = feature_image[:,:,1].ravel()
    color3 = feature_image[:,:,2].ravel()
    return np.hstack((color2, color3))

## Color Histogram

In [12]:
# Define a function to compute color histogram features(Use HSV and not use H channel.)
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately(S,V channels)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [13]:
#Show Random image about car and notcar image
ind_car = np.random.randint(0, len(cars))
ind_notcar = np.random.randint(0, len(notcars))

image_car = mpimg.imread(cars[ind_car])
image_notcar = mpimg.imread(notcars[ind_notcar])

pix_per_cell = 8
cell_per_block = 2
orient = 9

color_car = cv2.cvtColor(image_car, cv2.COLOR_RGB2HSV)
color_notcar = cv2.cvtColor(image_notcar, cv2.COLOR_RGB2HSV)
features_car, car_hog_image = get_hog_features(color_car[:,:,2], orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)
features_notcar, notcar_hog_image = get_hog_features(color_notcar[:,:,2], orient, pix_per_cell, cell_per_block, vis=True, feature_vec=False)
color_feat_car = convert_color_size(image_car, color_space='HSV')
color_feat_notcar = convert_color_size(image_notcar, color_space='HSV')

fig = plt.figure()
plt.subplot(441)
plt.imshow(image_car)
plt.title('Example Car Image')
plt.subplot(442)
plt.imshow(car_hog_image, cmap='gray')
plt.title('HOG Visualization Car')
plt.subplot(443)
plt.imshow(image_notcar)
plt.title('Example Not Car Image')
plt.subplot(444)
plt.imshow(notcar_hog_image, cmap='gray')
plt.title('HOG Visualization NotCar')
plt.subplot(445)
plt.imshow(color_car[:,:,0], cmap='gray')
plt.title('Car CH-1')
plt.subplot(446)
plt.imshow(color_feat_car[:,:,0], cmap='gray')
plt.title('Car CH-1 Features')
plt.subplot(447)
plt.imshow(color_notcar[:,:,0], cmap='gray')
plt.title('Not Car CH-1')
plt.subplot(448)
plt.imshow(color_feat_notcar[:,:,0], cmap='gray')
plt.title('NotCar CH-1 Features')
plt.subplot(449)
plt.imshow(color_car[:,:,1], cmap='gray')
plt.title('Car CH-2')
plt.subplot(4,4,10)
plt.imshow(color_feat_car[:,:,1], cmap='gray')
plt.title('Car CH-2 Features')
plt.subplot(4,4,11)
plt.imshow(color_notcar[:,:,1], cmap='gray')
plt.title('Not Car CH-2')
plt.subplot(4,4,12)
plt.imshow(color_feat_notcar[:,:,1], cmap='gray')
plt.title('NotCar CH-2 Features')
plt.subplot(4,4,13)
plt.imshow(color_car[:,:,2], cmap='gray')
plt.title('Car CH-3')
plt.subplot(4,4,14)
plt.imshow(color_feat_car[:,:,2], cmap='gray')
plt.title('Car CH-3 Features')
plt.subplot(4,4,15)
plt.imshow(color_notcar[:,:,2], cmap='gray')
plt.title('Not Car CH-3')
plt.subplot(4,4,16)
plt.imshow(color_feat_notcar[:,:,2], cmap='gray')
plt.title('NotCar CH-3 Features')
fig.tight_layout(pad = 0.1)

## Extract Feature

In [14]:
def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256)):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else:
            feature_image = np.copy(image)      
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        #Apply hog_feature() to use HOG algo.
        ch3 = feature_image[:,:,2]
        hog_features = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False).ravel()
        
        
        
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features, hog_features)))
    # Return list of feature vectors
    return features

## Normalization

In [116]:
from sklearn.preprocessing import StandardScaler

car_features = extract_features(cars, cspace='HSV', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256))
notcar_features = extract_features(notcars, cspace='HSV', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256))

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)
car_ind = np.random.randint(0, len(cars))
# Plot an example of raw and scaled features
fig = plt.figure(figsize=(12,4))
plt.subplot(131)
plt.imshow(mpimg.imread(cars[car_ind]))
plt.title('Original Image')
plt.subplot(132)
plt.plot(X[car_ind])
plt.title('Raw Features')
plt.subplot(133)
plt.plot(scaled_X[car_ind])
plt.title('Normalized Features')
fig.tight_layout()

## Linear SVMs Classifier

In [292]:
from sklearn.svm import SVC
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

rand_state = np.random.randint(0, 50)
Set_X, Set_y = shuffle(scaled_X, y)
X_train, X_test, y_train, y_test = train_test_split(Set_X, Set_y, test_size=0.2, random_state=rand_state)

# Use a linear SVC 
svc = SVC(kernel = 'linear', C = 40)
svc.fit(X_train, y_train)

print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
n_predict = 20
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])

Test Accuracy of SVC =  0.9892
My SVC predicts:  [ 0.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  0.  0.  0.  0.
  1.  0.]
For these 20 labels:  [ 0.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  0.  0.  0.  0.
  0.  0.]


## Delete False Positives

In [364]:
from scipy.ndimage.measurements import label

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image(Delete small false positives)
        if (bbox[1][0] - bbox[0][0]) >= 64 and (bbox[1][1] - bbox[0][1]) >= 64:
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

## Pipe Line

In [377]:
def find_cars(img, ystart, ystop, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    draw_img = np.copy(img)
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    box_list = []
    
    for i in range(2, 7):
        if i == 2:
            ystop = ystop - 64
        if i == 5:
            ystop = ystop + 64
            ystart = ystart + 64
            # 1, 1.5, 2, 2.5, 3 scales
        scale = i/2
        img_tosearch = img[ystart:ystop,:,:]
        ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2HSV)
    
    
        if scale != 1:
            imshape = ctrans_tosearch.shape
            ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
        ch3 = ctrans_tosearch[:,:,2]

        # Define blocks and steps as above
        nxblocks = (ch3.shape[1] // pix_per_cell)-1
        nyblocks = (ch3.shape[0] // pix_per_cell)-1 
        nfeat_per_block = orient*cell_per_block**2
        # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
        window = 64
        nblocks_per_window = (window // pix_per_cell)-1 
        cells_per_step = 2  # Instead of overlap, define how many cells to step
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
        # Compute individual channel HOG features for the entire image
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    
        for xb in range(nxsteps):
            for yb in range(nysteps):
                ypos = yb*cells_per_step
                xpos = xb*cells_per_step
                # Extract HOG for this patch
                hog_features = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()

                xleft = xpos*pix_per_cell
                ytop = ypos*pix_per_cell

                # Extract the image patch
                subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
                # Get color features
                spatial_features = bin_spatial(subimg, size=spatial_size)
                hist_features = color_hist(subimg, nbins=hist_bins)

                # Scale features and make a prediction
                test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
                #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
                test_prediction = svc.predict(test_features)
            
                if test_prediction == 1:
                    xbox_left = np.int(xleft*scale)
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(window*scale)                    
                    box_list.append([(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)])
                    
    heat = add_heat(heat,box_list)
    heat = apply_threshold(heat,2)
    heatmap = np.clip(heat, 0, 255)
    
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(draw_img , labels)
    return draw_img

In [378]:
ystart = 400
ystop = 656

test_image1 = mpimg.imread('./test_images/test1.jpg')
test_image2 = mpimg.imread('./test_images/test2.jpg')
test_image3 = mpimg.imread('./test_images/test3.jpg')
test_image4 = mpimg.imread('./test_images/test4.jpg')
test_image5 = mpimg.imread('./test_images/test5.jpg')
test_image6 = mpimg.imread('./test_images/test6.jpg')
out_img1 = find_cars(test_image1, ystart, ystop, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size = (32,32), hist_bins = 32)
out_img2 = find_cars(test_image2, ystart, ystop, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size = (32,32), hist_bins = 32)
out_img3 = find_cars(test_image3, ystart, ystop, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size = (32,32), hist_bins = 32)
out_img4 = find_cars(test_image4, ystart, ystop, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size = (32,32), hist_bins = 32)
out_img5 = find_cars(test_image5, ystart, ystop, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size = (32,32), hist_bins = 32)
out_img6 = find_cars(test_image6, ystart, ystop, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size = (32,32), hist_bins = 32)

In [386]:
fig = plt.figure()
plt.subplot(421)
plt.imshow(out_img1)
plt.title('TEST1')
plt.subplot(422)
plt.imshow(out_img2)
plt.title('TEST2')
plt.subplot(423)
plt.imshow(out_img3)
plt.title('TEST3')
plt.subplot(424)
plt.imshow(out_img4)
plt.title('TEST4')
plt.subplot(425)
plt.imshow(out_img5)
plt.title('TEST5')
plt.subplot(426)
plt.imshow(out_img6)
plt.title('TEST6')
fig.tight_layout(pad = 0.1)

## Test on Video

In [368]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [375]:
def process_image(img):
    pix_per_cell = 8
    cell_per_block = 2
    orient = 9
    ystart = 400
    ystop = 656
    output = find_cars(img, ystart, ystop, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size = (32,32), hist_bins = 32)
    return output

In [376]:
test_output = 'test_output.mp4'
clip1 = VideoFileClip("test_video.mp4")
test_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time test_clip.write_videofile(test_output, audio=False)

[MoviePy] >>>> Building video test_output.mp4
[MoviePy] Writing video test_output.mp4


 97%|█████████▋| 38/39 [00:29<00:00,  1.29it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_output.mp4 

CPU times: user 37 s, sys: 40 ms, total: 37 s
Wall time: 31 s


In [380]:
project_output = 'project_output.mp4'
clip2 = VideoFileClip("project_video.mp4")
project_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

[MoviePy] >>>> Building video project_output.mp4
[MoviePy] Writing video project_output.mp4


100%|█████████▉| 1260/1261 [49:57<00:02,  2.35s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_output.mp4 

CPU times: user 55min 1s, sys: 2.31 s, total: 55min 4s
Wall time: 49min 58s
